In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import cv2
import random
import pickle
import itertools

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import optimizers
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix

%matplotlib inline


In [ ]:
def plot_results(mod_history, metric, epochs):
      
      # Check out our train loss and test loss over epochs.
      train_metric = mod_history.history[metric]
      val = 'val_' + metric
      test_metric = mod_history.history[val]

      # Set figure size.
      plt.figure(figsize=(12, 8))

      # Generate line plot of training, testing loss over epochs.
      plt.plot(train_metric, label=f'Training {metric}', color='#185fad')
      plt.plot(test_metric, label=f'Testing {metric}', color='orange')

      # Set title
      plt.title(f'Training and Testing {metric} by Epoch', fontsize = 25)
      plt.xlabel('Epoch', fontsize = 18)
      plt.ylabel('Categorical Crossentropy', fontsize = 18)
      plt.xticks(range(0,epochs,5), range(0,epochs,5))
      plt.legend(fontsize = 18);

In [ ]:
def make_predictions(mod_name, steps=20):
    preds = mod_name.predict(X_test,steps=steps)
    preds = preds.argmax(axis=-1)

    y_test_labels = np.argmax(y_test, axis=-1)

    cm = confusion_matrix(y_test_labels,preds)

    plot_confusion_matrix(cm, cm_plot_labels, normalize=True,
                          title='Face Shape Normalized')

    plt.show()



In [ ]:
path = "/content/drive/My Drive/AI/FaceShape Dataset/data/"
# print(os.listdir(path))

X_train = np.asarray(pickle.load(open(path + "X_train_rgb.pickle","rb")))
y_train = np.asarray(pickle.load(open(path + "y_train_rgb.pickle","rb")))
X_test = np.asarray(pickle.load(open(path + "X_test_rgb.pickle","rb")))
y_test = np.asarray(pickle.load(open(path + "y_test_rgb.pickle","rb")))

In [ ]:
print("Data Summary")
print("--------------------")
print(f"X_train shape {X_train.shape}")
print(f"y_train shape {y_train.shape}")
print("--------------------")
print(f"X_test shape {X_test.shape}")
print(f"y_test shape {y_test.shape}")

Data Summary
--------------------
X_train shape (4000, 224, 224, 3)
y_train shape (4000, 5)
--------------------
X_test shape (1000, 224, 224, 3)
y_test shape (1000, 5)


In [ ]:


path_vggface = '/content/drive/My Drive/AI/FaceShape Dataset/rcmalli_vggface_tf_notop_vgg16.h5'

In [ ]:


base_model = VGG16(input_shape=(224, 224, 3),  
                   include_top=False,  
                   weights=path_vggface)  

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
	
model_t1 = Sequential()

In [ ]:


x = layers.Flatten()(base_model.output)

x = layers.Dense(64, activation='relu')(x)   
x = layers.Dropout(0.5)(x)
x = layers.Dense(5, activation='softmax')(x)  

model_t1 = tf.keras.models.Model(base_model.input, x)



model_t1.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

model_t1.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True)

In [ ]:
datagen.fit(X_train)

In [ ]:
history_t1 = model_t1.fit(datagen.flow(X_train, y_train, batch_size=64), 
                          steps_per_epoch=len(X_train)/64, epochs=10, 
                          validation_data=(X_test, y_test))

Epoch 1/10
63/62 [==============================] - 2434s 39s/step - loss: 1.0335 - acc: 0.6003 - val_loss: 0.6069 - val_acc: 0.7960
Epoch 2/10
63/62 [==============================] - 2404s 38s/step - loss: 0.6703 - acc: 0.7598 - val_loss: 0.4732 - val_acc: 0.8490
Epoch 3/10
63/62 [==============================] - 2358s 37s/step - loss: 0.5592 - acc: 0.8100 - val_loss: 0.4171 - val_acc: 0.8630
Epoch 4/10
63/62 [==============================] - 2366s 38s/step - loss: 0.4702 - acc: 0.8462 - val_loss: 0.3672 - val_acc: 0.8810
Epoch 5/10
63/62 [==============================] - 2366s 38s/step - loss: 0.4104 - acc: 0.8602 - val_loss: 0.3486 - val_acc: 0.8840
Epoch 6/10
63/62 [==============================] - 2371s 38s/step - loss: 0.3946 - acc: 0.8695 - val_loss: 0.3294 - val_acc: 0.8890
Epoch 7/10
63/62 [==============================] - 2365s 38s/step - loss: 0.3641 - acc: 0.8808 - val_loss: 0.3141 - val_acc: 0.8970
Epoch 8/10
63/62 [==============================] - 2357s 37s/step - 

In [ ]:
model_t1.save("model.h5")